In [1]:
import hdbscan
import pandas as pd
from plotnine import *
from plotnine.data import *
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE

In [2]:
customers = pd.read_csv(open('kunden.csv'), usecols=[0,2,3,5], names=['mandant', 'anrede','plz','landisocode'])
customers.plz = customers.plz.str.slice(0, 2)

customers = pd.concat([customers,pd.get_dummies(customers['mandant'], prefix='mandant')],axis=1)
customers = pd.concat([customers,pd.get_dummies(customers['anrede'], prefix='anrede')],axis=1)
customers = pd.concat([customers,pd.get_dummies(customers['plz'], prefix='plz')],axis=1)
customers = pd.concat([customers,pd.get_dummies(customers['landisocode'], prefix='landisocode')],axis=1)

customers.drop(['mandant'],axis=1, inplace=True)
customers.drop(['anrede'],axis=1, inplace=True)
customers.drop(['plz'],axis=1, inplace=True)
customers.drop(['landisocode'],axis=1, inplace=True)

customers = customers.sample(25000)

In [ ]:
pcaclusterer = hdbscan.HDBSCAN(algorithm='boruvka_kdtree', core_dist_n_jobs=6, alpha=1.0, approx_min_span_tree=True,
                               gen_min_span_tree=False, leaf_size=40,
                               metric='euclidean', min_cluster_size=120, min_samples=1, p=None)
pcaclusterer.fit(customers)


/usr/local/lib64/python3.7/site-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib64/python3.7/site-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib64/python3.7/site-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib64/python3.7/site-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)


In [ ]:
print("Anzahl der gefundenen Cluster: {}".format(pcaclusterer.labels_.max()))

In [ ]:
customers['cluster'] = pcaclusterer.labels_.astype(str)
customers.to_csv('kunden_clustrerassignment.csv')

In [ ]:
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(customers)

customers['x-tsne'] = tsne_results[:,0]
customers['y-tsne'] = tsne_results[:,1]

print(ggplot(aes(x='x-tsne', y='y-tsne', color="cluster"), data=customers) + labs(title = 'Cluster visualization') + geom_point(size=1,alpha=0.3))